In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import re
import json
import time

In [2]:
# Chrome
options = Options()
options.add_argument('--headless')
# Chromeを起動
driver = webdriver.Chrome(options=options)

In [23]:
# 指定のURLを開く
driver.get("https://iryoujyouhou.wiki.fc2.com/wiki/2024%3E%E5%8C%BB%E5%AD%A6%E5%8C%BB%E7%99%82%E7%B3%BB%3E%E5%95%8F%E9%A1%8C%E3%81%A8%E8%A7%A3%E8%AA%AC%3E46")

In [24]:
##非表示要素を表示する
driver.find_element(By.XPATH, '//div[@class="region"]/p/span').click()

## 問題文を取得

In [25]:
## 問題文を取得
elements = driver.find_element(By.XPATH, '//div[@class="user_body"]/div/span')
question = elements.text
extract_question_num_pattern = r"^問\d+"
exclude_question_pattern = r"^問\d+$"
question_num = re.findall(extract_question_num_pattern, question)[0]
if re.fullmatch(exclude_question_pattern, question):
    elements = driver.find_elements(By.XPATH, '//div[@class="user_body"]/div')
    question = elements[1].text
else:
    question = re.sub(extract_question_num_pattern, "", question)

print(f"{question_num}, {question}")

問46, 連続変量にあたるのはどれか。(解2)


## 選択肢を取得

In [26]:

elements = driver.find_elements(By.XPATH, '//div[@class="user_body"]/div')
len(elements)

10

In [27]:
pattern = r"^\d+\)"

In [28]:
select_text_list = []
for el in elements:
    if re.search(pattern, el.text):
        if re.search("解答", el.text):
            break
        else:
            print(el.text)
            select_text = re.sub(pattern, "", el.text)
            select_text = re.sub("　", " ", select_text) 
            select_text = re.sub(" ", "", select_text) 
            select_text_list.append(select_text)
print(select_text_list)

1) 血圧
2) 身長
3) 性別
4) 健康状態
5) 家族の人数
['血圧', '身長', '性別', '健康状態', '家族の人数']


## 解答を取得

In [10]:
## 回答を取得
elements = driver.find_elements(By.XPATH, '//div[@class="region"]/div/div')
len(elements)

7

In [11]:
pattern = r"解答】"
for el in elements:
    el_text = el.text
    if re.search(pattern, el_text):
        answer = el.text
        answer = re.sub(pattern, "", answer)
        answer = re.sub("　", "", answer)
        print(answer)

5


## jsonに保存

In [67]:
question_json = {
    "question": "問題1",
    "selects": select_text_list,
    "answer": "1"
}

In [68]:
question_json

{'question': '問題1',
 'selects': ['疾病予防 ー 一次予防',
  '早期発見 ー 一次予防',
  '再発予防 ー 二次予防',
  '事故予防 ー 二次予防',
  'リハビリテーション ー 二次予防'],
 'answer': '1'}

In [69]:
with open('test.json', 'w', encoding="utf-8") as f:
    json.dump(question_json, f, ensure_ascii=False)

In [33]:
# ブラウザを閉じる
driver.quit()

In [18]:
import json